In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# housing data import and add bias ahead of matrix
housing = fetch_california_housing()
m, n = housing.data.shape

In [2]:
n_epochs = 10000
learning_rate = 0.01

In [3]:
scaled_housing_data = housing.data / housing.data.max(axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
#scaled_housing_data_plus_bias

In [4]:
%%time
## check with https://github.com/tensorflow/tensorflow/issues/37144
tf.autograph.set_verbosity(10, True)

## tensorflow model
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

## random support of theta
theta = tf.Variable(tf.random.normal([n+1, 1], -1.0, 1.0), name="theta")

### tensorflow 2.0, refer to https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate.ipynb and https://www.tensorflow.org/guide/effective_tf2
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/function.ipynb#scrollTo=SbtT1-Wm70F2
# https://www.tensorflow.org/guide/autodiff
# converting
# @tf.function
# @tf.compat.v2.autograph.experimental.do_not_convert
def train(theta):
    y_pred = tf.matmul(X, theta, name="preditions")
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")
    gradients = 2/m * tf.matmul(tf.transpose(X), error)
    theta.assign(theta - learning_rate * gradients)
    return mse

for epoch in range(n_epochs+1):
    mse = train(theta)
    if epoch % 1000 == 0:
        # https://stackoverflow.com/questions/58441514/why-is-tensorflow-2-much-slower-than-tensorflow-1
        print("Epoch {}, MSE = {}".format(epoch, mse))

print("best beta : {}".format(theta))

Epoch 0, MSE = 80.15167236328125
Epoch 1000, MSE = 1.17268705368042
Epoch 2000, MSE = 0.9282985329627991
Epoch 3000, MSE = 0.796195924282074
Epoch 4000, MSE = 0.7246407866477966
Epoch 5000, MSE = 0.6857902407646179
Epoch 6000, MSE = 0.664614737033844
Epoch 7000, MSE = 0.6529957056045532
Epoch 8000, MSE = 0.6465469002723694
Epoch 9000, MSE = 0.6428980231285095
Epoch 10000, MSE = 0.6407684087753296
best beta : <tf.Variable 'theta:0' shape=(9, 1) dtype=float32, numpy=
array([[ 0.08040578],
       [ 6.140441  ],
       [ 0.8022814 ],
       [-1.9742593 ],
       [-0.5143442 ],
       [-1.0119156 ],
       [-2.1868894 ],
       [-1.6233538 ],
       [ 1.4138998 ]], dtype=float32)>
CPU times: user 6.66 s, sys: 448 ms, total: 7.1 s
Wall time: 7.11 s
